Importiamo le librerie necessarie per l'addestramento del classificatore

In [ ]:
from joblib import load
from pandas import DataFrame, read_csv, ExcelWriter
from os import chdir, listdir
from sys import path
from nltk import download
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from Addestramento_Classificatore import text_preprocessing
import matplotlib.pyplot as plt

Ci spostiamo nella directory in cui viene eseguito il programma.

In [ ]:
chdir(path[0])

Carichiamo il classificatore e gli altri oggetti necessari.

In [ ]:
clf = load('./Objects/clf.pkl')
count_vect = load('./Objects/count_vect.pkl')
tfidf_transformer = load('./Objects/tfidf_transformer.pkl')

Eseguiamo il download del file per il SentimentIntesityAnalyzer() di nltk e ne creiamo un oggetto, insieme ad una lista da utilizzare dopo.


In [ ]:
download('vader_lexicon')
sa = SentimentIntensityAnalyzer()

movies_results = []

Il ciclo for dovrà analizzare tutti i file contenuti nella cartella *Reviews*

In [ ]:
for file in listdir('./Reviews'):
    pass

Creiamo il dataframe leggendo i file .csv

In [ ]:
    df = read_csv(f'./Reviews/{file}')

Applichiamo il preprocessing a tutte le recensioni.

In [ ]:
    df['Reviews'] = df['Reviews'].map(text_preprocessing)

Come già fatto per il test del classificatore eseguiamo questi altri test,

In [ ]:
    X_test_matrix = count_vect.transform(df['Reviews'])
    X_test_tfidf = tfidf_transformer.transform(X_test_matrix)

    predicted = clf.predict(X_test_tfidf)

Otteniamo il numero delle recensioni andando a prendere il numero di elementi in una colonna, la prima in questo caso ma non ha importanza. Hanno infatti tutte lo stesso numero di elementi.

In [ ]:
    number_reviews = df.count()[0]

Facciamo una semplice media con tutti gli i voti delle recensioni IMDb.

In [ ]:
    average = 0
    for item in df['Rating']:
        average += item
    average /= number_reviews

Convertiamo il numpy array in una tupla e contiamo gli elementi che corrispondono alla stringa **'positive'**.
Successivamente il tasso positivo è calcolato come il rapporto tra i positivi e il numero totale, poi passato in percentuale.

In [ ]:
    positives = tuple(predicted).count('positive')
    positive_rate = (positives/number_reviews)*100

Ci occupiamo ora del tasso di positività da ottenere con il SentimentIntesityAnalyzer() di nltk.
Passiamo ognuno delle recensioni al metodo *polarity_scores*, che restituisce un dizionario formato che conterrà le chiavi relative al valore positivo, neutro, negativo e composto. Siccome stiamo analizzando solo i positivi, al dizionario ottenuto tramite il metodo applico la lettura del valore tramite chiave **'pos'**. Il resto è una semplice media passata poi in percentuale.

In [ ]:
    positive_rate_nltk = 0
    for item in df['Reviews']:
        positive_rate_nltk += sa.polarity_scores(item)['pos']
    positive_rate_nltk = (positive_rate_nltk/number_reviews)*100

Creo un dizionario con le informazioni necessarie all'analisi. Per quanto riguarda le informazioni numeriche abbiamo inserito la funzione *round* per approssimare a due cifre dopo la virgola.
Successivamente inseriamo il dizionario nella lista che contenenti i dizionari di tutti i film.

In [ ]:
    dictionary = {
        'Movie' : file.replace('.csv', ''),
        'Rating' : f'{round(average, 2)}/10',
        'Reviews' : number_reviews,
        'Positive rate' : f'{round(positive_rate, 2)}%',
        'Positive rate nltk' : f'{round(positive_rate_nltk, 2)}%'
    }

    movies_results.append(dictionary)

Finito il ciclo for, creiamo un *dataframe* con i risultati ottenuti.

In [ ]:
df = DataFrame(movies_results)

Sfruttando la libreria *xlsxwriter* creiamo anche un file excel in cui esportare i risultati.

In [ ]:
excel_file = ExcelWriter('./Results/Results_Sorted.xlsx', engine='xlsxwriter')

Ordino il dataframe in tre modi:
1. In maniera decrescente per *voti* delle recensioni IMDb.
2. In maniera decrescente per *tasso di positività* del nostro classificatore
3. In maniera decrescente per *tasso di positività* del SentimentIntesityAnalyzer() di nltk

Tutti e tre vengono stampati a schermo e inseriti in un foglio di calcolo Excel uno di fianco all'altro.

In [ ]:
df.sort_values(by = 'Rating', ascending=False)
df.to_excel(excel_file, sheet_name='Marvel_Analysis')
print(df)

df.sort_values(by = 'Positive rate', ascending=False)
df.to_excel(excel_file, sheet_name='Marvel_Analysis', startcol=9)
print(df)

df.sort_values(by = 'Positive rate nltk', ascending=False)
df.to_excel(excel_file, sheet_name='Marvel_Analysis', startcol=18)
print(df)

Infine salviamo il file excel.

In [ ]:
excel_file.save()